In [ ]:
from collections.abc import KeysView
import numpy as np
import matplotlib.pyplot as plt


def gillespie_2_type_pop(Nit, fA, gA, NA0, fB, gB, NB0, K, tlist, alpha, beta, gamma):
    NAlist = np.full((len(tlist), Nit), np.nan)
    NBlist = np.full((len(tlist), Nit), np.nan)

    for i in range(Nit):
        # Initialization
        NA = NA0  # Initialization of the number of A individuals
        NB = NB0  # Initialization of the number of B individuals
        t = 0     # Initialization of time
        q = 1     # Index for the vector of time points
        NAlist[q - 1, i] = NA  # Vector of the population size of A individuals
        NBlist[q - 1, i] = NB  # Vector of the population size of B individuals
        q += 1
        cumul = np.zeros(4)  # To build the sampling tower

        while (NA != 0 and NB != 0) and t < max(tlist):
            # Compute the transition rates
            TrepA = fA * (np.log(K/(NA+NB))) * NA**2
            TrepB = fB * (np.log(K/(NA+NB)))* NB**2
            TdeathA = gA * NA
            TdeathB = gB * NB
            T = TrepA + TrepB + TdeathA + TdeathB

            # Update time
            r1 = np.random.rand()
            tau = 1 / T * np.log(1 / r1)
            t = t + tau

            while t > tlist[q - 1] and q < len(tlist):
                NAlist[q - 1, i] = NA
                NBlist[q - 1, i] = NB
                q += 1

            # Build a sampling tower
            ir2 = 1
            r2 = np.random.rand()
            cumul[0] = TrepA
            cumul[1] = TrepA + TrepB
            cumul[2] = TrepA + TrepB + TdeathA
            cumul[3] = TrepA + TrepB + TdeathA + TdeathB

            # Determine which reaction occurs and update the number of individuals
            while cumul[ir2 - 1] < r2 * T:
                ir2 += 1

            if ir2 == 1:
                NA += 1
            elif ir2 == 2:
                NB += 1
            elif ir2 == 3:
                NA -= 1
            elif ir2 == 4:
                NB -= 1

        while q <= len(tlist):
            NAlist[q - 1, i] = NA
            NBlist[q - 1, i] = NB
            q += 1

    return NAlist, NBlist
    import matplotlib.pyplot as plt

# Set parameters
Nit = 1000
fA, gA, NA0 = 1, 0.1, 90
fB, gB, NB0 = 1.01, 0.1, 1
alpha=1
beta=2
gamma=1.5
K = 200
tlist = np.linspace(0, 100, 100)

# Run the Python function
NAlist, NBlist = gillespie_2_type_pop(Nit, fA, gA, NA0, fB, gB, NB0, K, tlist, alpha, beta, gamma)

# Plot the results
for i in range(Nit):
    plt.plot(tlist, NAlist[:, i], label=f'Run {i+1} - Type A')
    plt.plot(tlist, NBlist[:, i], label=f'Run {i+1} - Type B')

plt.xlabel('Time')
plt.ylabel('Population Size')
plt.show()

survival_count_B = np.sum(NBlist[-1, :] > 0)
print(f"Number of times Type B survives: {survival_count_B} out of {Nit} simulations.")

# Vary fB values
f=fB_values = np.linspace(1.01, 2, 20)
survival_counts = []
function_values = []
functional_values = []
f2_values =[]
for fB in fB_values:
    # Run the Python function
    NAlist, NBlist = gillespie_2_type_pop(Nit, fA, gA, NA0, fB, gB, NB0, K, tlist, alpha,beta, gamma)

    # Count the number of times Type B survives
    survival_count_B = np.sum(NBlist[-1, :] > 0)/1000
    survival_counts.append(survival_count_B)

    Keff=K*np.exp(-gA/fA)
    factor=21

for fB in fB_values_2:
    # Calculate the corrected function value using only the last element
    functional_value = (1-fA*gB/(fB*gA))/(1-(fA*gB/(fB*gA))**Keff)
    functional_values.append(functional_value)



for fB in fB_values_2:
    # Calculate the corrected function value using only the last element
    function_value = (1-fA*gB/(fB*gA))/(1-(fA*gB/(fB*gA))**Keff)
    function_values.append(function_value)


# Plot both on the same graph
plt.figure(figsize=(10, 6))

# Plot survival_count_B and function_values against fB on the same y-axis
plt.plot(fB_values, survival_counts, marker='o', color='b', label='Survival Count for Type B w Allee effect')

plt.xlabel('Fitness of mutant')
plt.ylabel('Fixation Probability')
plt.ylim (0,1)
plt.title('Survival Count and Function vs. gB')
plt.legend()
plt.show()

lowest_NB0_values = []

# Loop over different fB values
for fB in fB_values:
    # Initialize NB0 to None (if no non-zero fixation probability is found)
    lowest_NB0 = None

    # Iterate over a range of NB0 values
    for NB0 in range(1, 100):
        # Run the simulation
        NAlist, NBlist = gillespie_2_type_pop(Nit, fA, gA, NA0, fB, gB, NB0, K, tlist, alpha, beta, gamma)

        # Calculate the fixation probability for Type B
        fixation_probability_B = np.mean(NBlist[-1, :] > 0)

        # If fixation probability is non-zero, update lowest_NB0 and break the loop
        if fixation_probability_B > 0.01:
            lowest_NB0 = NB0
            break

    # Append the lowest NB0 for the current fB to the list
    lowest_NB0_values.append(lowest_NB0)

# Plot the results
plt.plot(fB_values, lowest_NB0_values, marker='o', color='pink')
plt.xlabel('Birth rate of Type B individuals (fB)')
plt.ylabel('Lowest initial population size of Type B (NB0)')
plt.title('Lowest NB0 for Each Value of fB')
plt.grid(True)
plt.show()